In [1]:
# CELL 1 — imports & setup
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
from tqdm import tqdm

# Text / NLP
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# download nltk stopwords first-run
nltk.download('stopwords')

DATA_PATH = Path("../data/raw/raw_analyst_ratings.csv")  # your file
OUT_DIR = Path("../data/processed/eda_outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

pd.set_option('display.max_columns', 120)


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
import nltk
nltk.download("stopwords")


In [ ]:
# CELL 2 — robust CSV loader (handles very large files)
def load_csv_memory_friendly(path, sample_frac=None, nrows=None, chunksize=200_000):
    """
    - If sample_frac provided, returns a random sample (read in chunks).
    - If nrows provided, reads only first nrows.
    - Otherwise attempts fast full read.
    """
    path = Path(path)
    if nrows is not None:
        df = pd.read_csv(path, nrows=nrows)
        return df

    # quick attempt to read full file (fast path)
    try:
        df = pd.read_csv(path)
        return df
    except MemoryError:
        # fallback: sample rows streaming
        if sample_frac is None:
            raise MemoryError("File too large to load in memory; provide sample_frac.")
        rng = np.random.default_rng(0)
        chunks = []
        for chunk in pd.read_csv(path, chunksize=chunksize):
            mask = rng.random(chunk.shape[0]) < sample_frac
            if mask.any():
                chunks.append(chunk.loc[mask])
        if chunks:
            return pd.concat(chunks, ignore_index=True)
        else:
            return pd.DataFrame(columns=chunk.columns)

# Use it:
df = load_csv_memory_friendly(DATA_PATH, sample_frac=None, nrows=None)
print("Loaded rows:", len(df))


In [ ]:
# CELL 3 — inspect columns and sample rows
print(df.columns.tolist())
display(df.head(5))


In [ ]:
# CELL 4 — normalize column names and basic cleanup
# Based on your sample header: headline,url,publisher,date,stock
expected = ['headline','url','publisher','date','stock']
# normalize columns (lowercase, strip)
df.columns = [c.strip().lower() for c in df.columns]

# Keep only expected columns if extras exist
keep = [c for c in expected if c in df.columns]
df = df[keep].copy()

# drop exact duplicates (same url or same headline + date)
if 'url' in df.columns:
    df = df.drop_duplicates(subset=['url'])
else:
    df = df.drop_duplicates()

# Basic info
print("Rows after dedup:", len(df))
df.info()


In [ ]:
# CELL 5 — parse dates with timezone handling
# Your sample date looks like: 2020--05 10:30:54-04:00  (typo??) — we'll try flexible parsing
def safe_parse_datetime(s):
    try:
        return pd.to_datetime(s, utc=True, errors='coerce')
    except Exception:
        return pd.to_datetime(s, errors='coerce')

df['date_parsed'] = df['date'].astype(str).apply(safe_parse_datetime)
# If parsing produced naive timestamps, make them UTC-aware; assume input already has offset per description.
# Report parsing issues
n_bad = df['date_parsed'].isna().sum()
print(f"Failed to parse {n_bad} date rows out of {len(df)}")
# If many fails, show some examples
if n_bad:
    display(df.loc[df['date_parsed'].isna(), ['date']].head(10))

# create common columns for analysis
df['date_utc'] = df['date_parsed'].dt.tz_convert('UTC')   # all to UTC
df['date_local'] = df['date_parsed']                     # original tz if present
df['date'] = df['date_utc'].dt.normalize()               # trading-day date (midnight UTC)
df['year'] = df['date_utc'].dt.year
df['month'] = df['date_utc'].dt.month
df['day'] = df['date_utc'].dt.day
df['hour'] = df['date_utc'].dt.hour
df['weekday'] = df['date_utc'].dt.day_name()


In [ ]:
# CELL 6 — Descriptive statistics: headline lengths & NAs
df['headline'] = df['headline'].astype(str)
df['headline_len'] = df['headline'].str.len().fillna(0).astype(int)

print("Headline length stats:\n", df['headline_len'].describe())

# Nulls per column
null_summary = df.isna().sum().to_frame('n_nulls')
null_summary['pct_null'] = null_summary['n_nulls'] / len(df) * 100
display(null_summary)


In [ ]:
# plot headline length distribution
plt.figure(figsize=(8,4))
sns.histplot(df['headline_len'], bins=50)
plt.title("Headline length distribution")
plt.xlabel("length (chars)")
plt.ylabel("count")
plt.tight_layout()
plt.savefig(OUT_DIR/"headline_length_hist.png", dpi=150)
plt.show()


In [ ]:
# CELL 7 — Publisher counts (top publishers)
publisher_counts = df['publisher'].value_counts(dropna=True)
display(publisher_counts.head(30))

plt.figure(figsize=(8,10))
publisher_counts.head(30).sort_values().plot(kind='barh')
plt.title("Top 30 publishers by article count")
plt.xlabel("count")
plt.tight_layout()
plt.savefig(OUT_DIR/"top_publishers.png", dpi=150)
plt.show()


In [ ]:
# CELL 8 — Publication trends over time (articles per day)
daily_counts = df.groupby('date').size().rename('n_articles').reset_index()
daily_counts = daily_counts.sort_values('date')

plt.figure(figsize=(12,4))
plt.plot(daily_counts['date'], daily_counts['n_articles'])
plt.title("Articles per day (time series)")
plt.xlabel("date")
plt.ylabel("n_articles")
plt.tight_layout()
plt.savefig(OUT_DIR/"articles_per_day.png", dpi=150)
plt.show()

# show peaks
daily_counts.nlargest(10, 'n_articles')


In [ ]:
# CELL 9 — Time-of-day and weekday distributions
plt.figure(figsize=(10,4))
sns.countplot(x='hour', data=df, order=range(0,24))
plt.title("Articles by hour of day (UTC)")
plt.savefig(OUT_DIR/"articles_by_hour.png", dpi=150)
plt.show()

plt.figure(figsize=(8,4))
order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
sns.countplot(x='weekday', data=df, order=order)
plt.title("Articles by weekday")
plt.savefig(OUT_DIR/"articles_by_weekday.png", dpi=150)
plt.show()


In [ ]:
# CELL 10 — Text analysis: cleaning & tokens
STOPWORDS = set(stopwords.words('english'))
WORD_RE = re.compile(r'\b[a-z]{2,}\b')

def tokens(text):
    if not isinstance(text, str): return []
    text = text.lower()
    text = re.sub(r'http\S+', ' ', text)  # remove urls
    toks = WORD_RE.findall(text)
    toks = [t for t in toks if t not in STOPWORDS]
    return toks

# build token frequency (sample to speed up if large)
SAMPLE_N = 200_000
iter_df = df['headline']
if len(df) > SAMPLE_N:
    iter_df = df['headline'].sample(SAMPLE_N, random_state=0)

all_tokens = []
for t in tqdm(iter_df, desc="tokenizing"):
    all_tokens.extend(tokens(t))

most_common = Counter(all_tokens).most_common(60)
print("Top words:", most_common[:30])


In [ ]:
# CELL 11 — Bigrams (frequent phrases)
from nltk import ngrams

def top_bigrams(headlines, top_k=40):
    c = Counter()
    for h in headlines:
        ts = tokens(h)
        for bg in ngrams(ts, 2):
            c[' '.join(bg)] += 1
    return c.most_common(top_k)

bigrams_top = top_bigrams(iter_df, top_k=50)
print("Top bigrams:", bigrams_top[:30])


In [ ]:
# CELL 12 — Lightweight topic modeling (TF-IDF + KMeans)
# We'll vectorize headlines and cluster them. Adjust n_clusters as needed.
SAMPLE_TEXTS = df['headline'].dropna().sample(min(20000, len(df)), random_state=0).tolist()

tfv = TfidfVectorizer(max_df=0.8, min_df=10, ngram_range=(1,2), max_features=5000)
X = tfv.fit_transform(SAMPLE_TEXTS)

n_clusters = 12
km = KMeans(n_clusters=n_clusters, random_state=0, n_init=10)
km.fit(X)
labels = km.labels_

# top terms per cluster
terms = tfv.get_feature_names_out()
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
cluster_terms = {}
for i in range(n_clusters):
    cluster_terms[i] = [terms[ind] for ind in order_centroids[i, :20]]
    print(f"Cluster {i} top terms:", cluster_terms[i][:10])


In [ ]:
# CELL 13 — Save processed data & summary CSVs for later work
df.to_parquet(OUT_DIR/"news_processed.parquet", index=False)
pd.DataFrame(most_common, columns=['word','count']).to_csv(OUT_DIR/"top_words.csv", index=False)
pd.DataFrame(bigrams_top, columns=['bigram','count']).to_csv(OUT_DIR/"top_bigrams.csv", index=False)
daily_counts.to_csv(OUT_DIR/"daily_counts.csv", index=False)
publisher_counts.to_csv(OUT_DIR/"publisher_counts.csv", header=['count'])
print("Saved outputs to", OUT_DIR)
